In [78]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn import svm
from sklearn import model_selection
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.metrics import sensitivity_specificity_support
#from scipy import stats
import math
import statistics as stats

In [79]:
df1 = pd.read_csv("HurstExp.csv")
df1 = df1.sort_values(by='ID')
row_labels = df1.get('ID').values
df1 = df1.drop(['ID'], axis=1)
column_labels = df1.columns.values

df1 = pd.DataFrame(data=df1.values, index=row_labels, columns=column_labels, dtype=float)

EDSS = df1['EDSS']
data1 = df1.drop(['EDSS'],axis=1)
data1.head()
#print(df.shape)
#df.head()

,Left-Cerebellum-Cortex,Left-Thalamus-Proper,Left-Caudate,Left-Putamen,Left-Pallidum,Left-Hippocampus,Left-Amygdala,Left-Accumbens-area,Left-VentralDC,Right-Cerebellum-Cortex,...,ctx-rh-rostralanteriorcingulate,ctx-rh-rostralmiddlefrontal,ctx-rh-superiorfrontal,ctx-rh-superiorparietal,ctx-rh-superiortemporal,ctx-rh-supramarginal,ctx-rh-frontalpole,ctx-rh-temporalpole,ctx-rh-transversetemporal,ctx-rh-insula
MS0001_20190107.txt,0.963565,1.151747,0.640621,1.133392,0.920361,0.816062,0.881586,1.042133,1.163728,0.926568,...,0.749424,0.729738,0.822625,0.796755,1.119606,1.208903,1.060034,0.907452,0.713536,1.155636
MS0040_20180727.txt,0.935275,0.921987,0.870222,0.773119,0.504730,0.711951,0.853368,0.782612,1.333946,1.097145,...,0.652630,0.867170,0.894605,1.008555,0.907660,0.790810,1.462357,1.029638,1.070279,0.777443
MS0046_20170614.txt,0.894088,0.452763,0.565026,0.430652,0.590305,0.722065,0.900758,0.656344,0.865503,0.661753,...,0.860811,0.813552,0.667461,0.710838,0.724909,1.296127,1.105518,0.901974,0.740423,0.781980
MS0084_20171213.txt,1.057811,0.825563,0.783854,0.884995,0.758981,0.737332,0.690236,0.842655,0.754437,1.004131,...,0.782535,0.959180,1.014682,0.672343,0.894567,0.946074,1.028412,0.754140,1.110702,1.025216
MS0087_20180822.txt,0.673556,0.664157,0.605858,0.617874,0.919051,0.849110,0.922179,0.838192,0.692648,0.844454,...,0.873510,0.935600,0.892939,0.699367,0.982639,0.916412,0.884018,0.940411,0.731241,0.635838


In [80]:
#patient demographic info
df2 = pd.read_csv("Patient_info.csv")
df2 = df2.sort_values('MSID')
row_labels = df2.get('MSID').values
df2 = df2.drop(['MSID','Race','Ethnicity','Notes for MRI','MRI?','Spinal Lesion Number','FreeSurfer QC (YES = pass/No = fail)', 'Notes for FreeSurfer Editing','Notes for Spine'], axis=1)
column_labels = df2.columns.values

df2 = pd.DataFrame(data=df2.values, index=row_labels, columns=column_labels)

#EDSS2 = df2['EDSS']
data2 = df2.drop(['EDSS'],axis=1)


In [81]:
#categorize EDSS <=1.5 is low, >1.5 is high
n,d = data1.shape
#labels = np.where(EDSS<=1.5, 0, 1)
#labels = pd.DataFrame(labels, columns=['EDSS'])

for i in range(n):
    if EDSS[i]<=1.5:
        EDSS[i]=0
    else:
        EDSS[i]=1

for i in range(data2['Gender'].size):
    if data2['Gender'][i] == 'M':
        data2['Gender'][i] = 1
    else:
        data2['Gender'][i] = 0
    #if EDSS2[i]<=1.5:
    #    EDSS2[i]=0;
    #else EDSS2[i]=0
EDSS.head(20)
        

MS0001_20190107.txt    0.0
MS0040_20180727.txt    0.0
MS0046_20170614.txt    0.0
MS0084_20171213.txt    0.0
MS0087_20180822.txt    0.0
MS0101_20190109.txt    0.0
MS0105_20170927.txt    1.0
MS0106_20180912.txt    0.0
MS0108_20180410.txt    0.0
MS0131_20180719.txt    0.0
MS0159_20180202.txt    1.0
MS0160_20180601.txt    1.0
MS0190_20180606.txt    0.0
MS0204_20181120.txt    1.0
MS0241_20180221.txt    0.0
MS0266_20180212.txt    1.0
MS0278_20180420.txt    0.0
MS0282_20170519.txt    0.0
MS0288_20180615.txt    0.0
MS0397_20171213.txt    0.0
Name: EDSS, dtype: float64

In [5]:
def best_hp(params, auc):
    
    max_auc = np.amax(auc)
    imax = np.where(auc == max_auc)[0]
    num_max = len(imax)
    
    if num_max%2==0:
        index = np.argmax(auc)
        best_cost = params[index][0]
        best_gamma = params[index][1]
        max_auc = auc[index]
        return best_cost, best_gamma, max_auc

    best_Cs = [0]*num_max
    best_Gs = [0]*num_max
    
    for i in range(num_max):
        index = imax[i]
        best_Cs[i] = params[index][0]
        best_Gs[i] = params[index][1]
    
    try:
        best_cost = stats.mode(best_Cs) #if no unique mode, then smallest of the modes is chosen instead
    except:
        best_cost = stats.median(best_Cs)
        
    try:
        best_gamma = stats.mode(best_Gs) #if no unique mode, then smallest of the modes is chosen instead
    except:
        best_gamma = stats.median(best_Gs)
  
    
    return best_cost, best_gamma, max_auc



In [6]:
def svm_classify(X_train, y_train, X_test, y_test, C, gamma, kernel, cat):
    svm_model = svm.SVC(C=C, kernel=kernel, gamma=gamma, probability=True)
    #svm_bin = svm.SVC(C=C, kernel=kernel, gamma=gamma, probability=False)
    
    if not cat:
        ss=StandardScaler()
        ss.fit(X_train)
        X_train = ss.transform(X_train)
        sm=SMOTE()
        X_train, y_train = sm.fit_resample(X_train, y_train)
    
    #ss.fit(X_test)
        X_test = ss.transform(X_test)
    
    svm_model.fit(X_train, y_train)
    
    pred = svm_model.predict(X_test)
    pred_score = svm_model.predict_proba(X_test)
    
    curve = metrics.roc_curve(y_test, pred_score[:,1])
    #auc = calc_auc(curve)
    
    auc = metrics.roc_auc_score(y_test, pred_score[:,1], average=None)
    bal_acc = metrics.balanced_accuracy_score(y_test, pred)
    sens, spec, _ = sensitivity_specificity_support(y_test, pred, average='binary')
    
    return auc, bal_acc, sens, spec, pred_score[:,1], curve




In [77]:
k_fold = 5
kernel = 'linear'
outer = 20
inner = 5

cost = []
gamma = []
for i in range(-5,5):
    cost.append(10**i)
    #gamma.append(10**i)
    #gamma.append(10**i)
for i in range(-5, 0):
    gamma.append(10**i)
    
#print(gamma)

n,m = len(cost), len(gamma)
params = np.zeros((n*m, 2))

for i in range(n):
    for j in  range(m):
        params[i*m+j][0]=cost[i]
        params[i*m+j][1]=gamma[j]
        
Rkf = model_selection.RepeatedStratifiedKFold(n_splits=k_fold, n_repeats=outer)
kf = model_selection.StratifiedKFold(n_splits=k_fold, shuffle=True)




In [18]:
def run(data1, data2, target, cat):
    results1 = []
    results2 = []
    results3 = []
    #all_results = []
    #auc_curve = []
    
    #split entire data for 5-fold CV
    for train, test in Rkf.split(data1, target):
        X_train1 = data1.iloc[train, :]
        X_test1 = data1.iloc[test, :]
        X_train2 = data2.iloc[train, :]
        X_test2 = data2.iloc[test, :]
        
        y_train = target.iloc[train]
        y_test = target.iloc[test]
        
        inner_par1 = np.zeros((inner,2))
        inner_auc1 = np.zeros(inner)
        inner_par2 = np.zeros((inner,2))
        inner_auc2 = np.zeros(inner)
        
        #inner loop
        for y in range(inner):
            
            count=0
            all_auc1 = np.zeros((n*m, k_fold))
            all_auc2 = np.zeros((n*m, k_fold))
            
            #split for 5-fold CV on inner
            for tr, val in kf.split(X_train1, y_train):
                X_tr1 = X_train1.iloc[tr, :]
                X_val1 = X_train1.iloc[val, :]
                X_tr2 = X_train2.iloc[tr, :]
                X_val2 = X_train2.iloc[val, :]
                
                y_tr = y_train.iloc[tr]
                y_val = y_train.iloc[val]
                #print(y_tr)
                
                for p in range(n*m):
                    C=params[p][0]
                    G=params[p][1]
            
                    auc1, acc1, sen1, _, _, _ = svm_classify(X_tr1, y_tr, X_val1, y_val, C, G, kernel, cat)
                    auc2, acc2, sen2, _, _, _ = svm_classify(X_tr2, y_tr, X_val2, y_val, C, G, kernel, True)
                    all_auc1[p][count] = auc1
                    all_auc2[p][count] = auc2
                count += 1
            
            medians1 = np.median(all_auc1, axis=1)
            medians2 = np.median(all_auc2, axis=1)
            good_C1, good_G1, good_auc1 = best_hp(params, medians1)
            good_C2, good_G2, good_auc2 = best_hp(params, medians2)
            
            inner_par1[y][0] = good_C1
            inner_par1[y][1] = good_G1
            inner_auc1[y] = good_auc1
            inner_par2[y][0] = good_C2
            inner_par2[y][1] = good_G2
            inner_auc2[y] = good_auc2
            #all_results.append([good_C1, good_G1, good_auc1])
            
        best_C1, best_G1, best_auc1 = best_hp(inner_par1, inner_auc1)
        best_C2, best_G2, best_auc2 = best_hp(inner_par2, inner_auc2)
        
        total_auc1, bal_acc1, sens1, spec1, pred1, curve1 = svm_classify(X_train1, y_train, X_test1, y_test, best_C1, best_G1, kernel, cat)
        total_auc2, bal_acc2, sens2, spec2, pred2, curve2 = svm_classify(X_train2, y_train, X_test2, y_test, best_C2, best_G2, kernel, True)
        
        ens_pred_score = [0]*len(pred1)
        ens_preds = [0]*len(pred1)
        
        for i in range(len(pred1)):
            ens_pred_score[i] = (pred1[i] + pred2[i])/2
            if ens_pred_score[i]<.5:
                ens_preds[i] = 0
            else:
                ens_preds[i] = 1
            
        total_auc3 = metrics.roc_auc_score(y_test, ens_pred_score, average=None)
        bal_acc3 = metrics.balanced_accuracy_score(y_test, ens_preds)
        sens3, spec3, _ = sensitivity_specificity_support(y_test, ens_preds, average='binary')
        
        results1.append([best_C1, best_G1, total_auc1, bal_acc1, sens1, spec1])
        results2.append([best_C2, best_G2, total_auc2, bal_acc2, sens2, spec2])
        results3.append([total_auc3, bal_acc3, sens3, spec3])
        
    return results1, results2, results3

In [19]:
r1, r2, r3 = run(data1, data2, EDSS, False)




In [20]:
med_auc = np.median(r1, axis=0)[2]
med_bacc = np.median(r1, axis=0)[3]
med_sens = np.median(r1, axis=0)[4]
med_spec = np.median(r1, axis=0)[5]
print('Hurst:')
print('Median AUC is ' + str(med_auc))
print('Median Balanced Accuracy is ' + str(med_bacc))
print('Median Sensitivity is ' + str(med_sens))
print('Median Specificity is ' + str(med_spec))
res_df = pd.DataFrame(np.array(r1), columns=['Cost', 'Gamma', 'AUC', 'Balanced Accuracy', 'Sensitivity', 'Specificity'])
res_df

Hurst:
Median AUC is 0.4727272727272728
Median Balanced Accuracy is 0.5
Median Sensitivity is 0.0
Median Specificity is 0.9090909090909091


,Cost,Gamma,AUC,Balanced Accuracy,Sensitivity,Specificity
0,10.00000,0.00001,0.454545,0.463636,0.20,0.727273
1,0.10000,0.01000,0.500000,0.500000,0.00,1.000000
2,0.00010,0.01000,0.527273,0.500000,0.00,1.000000
3,100.00000,0.10000,0.725000,0.500000,0.00,1.000000
4,0.00010,0.00100,0.300000,0.700000,0.50,0.900000
5,100.00000,0.10000,0.381818,0.500000,0.00,1.000000
6,0.00001,0.00100,0.500000,0.463636,0.20,0.727273
7,0.00010,0.01000,0.381818,0.554545,0.20,0.909091
8,100.00000,0.00010,0.425000,0.525000,0.25,0.800000
9,1.00000,0.00001,0.475000,0.450000,0.50,0.400000


In [21]:
med_auc = np.median(r2, axis=0)[2]
med_bacc = np.median(r2, axis=0)[3]
med_sens = np.median(r2, axis=0)[4]
med_spec = np.median(r2, axis=0)[5]
print('Demographic:')
print('Median AUC is ' + str(med_auc))
print('Median Balanced Accuracy is ' + str(med_bacc))
print('Median Sensitivity is ' + str(med_sens))
print('Median Specificity is ' + str(med_spec))
res_df = pd.DataFrame(np.array(r2), columns=['Cost', 'Gamma', 'AUC', 'Balanced Accuracy', 'Sensitivity', 'Specificity'])
res_df

Demographic:
Median AUC is 0.5181818181818181
Median Balanced Accuracy is 0.5
Median Sensitivity is 0.0
Median Specificity is 1.0


,Cost,Gamma,AUC,Balanced Accuracy,Sensitivity,Specificity
0,100.00000,0.01000,0.400000,0.654545,0.40,0.909091
1,0.00010,0.00010,0.236364,0.500000,0.00,1.000000
2,0.01000,0.01000,0.509091,0.500000,0.00,1.000000
3,0.10000,0.00001,0.500000,0.500000,0.00,1.000000
4,0.01000,0.00001,0.650000,0.500000,0.00,1.000000
5,0.00001,0.00010,0.581818,0.500000,0.00,1.000000
6,0.00010,0.00100,0.981818,0.500000,0.00,1.000000
7,0.00100,0.00010,0.818182,0.500000,0.00,1.000000
8,0.00001,0.00100,0.900000,0.500000,0.00,1.000000
9,0.00001,0.00001,0.275000,0.500000,0.00,1.000000


In [22]:
med_auc = np.median(r3, axis=0)[0]
med_bacc = np.median(r3, axis=0)[1]
med_sens = np.median(r3, axis=0)[2]
med_spec = np.median(r3, axis=0)[3]
print('Ensemble')
print('Median AUC is ' + str(med_auc))
print('Median Balanced Accuracy is ' + str(med_bacc))
print('Median Sensitivity is ' + str(med_sens))
print('Median Specificity is ' + str(med_spec))
res_df = pd.DataFrame(np.array(r3), columns=['AUC', 'Balanced Accuracy', 'Sensitivity', 'Specificity'])
res_df

Ensemble
Median AUC is 0.4954545454545455
Median Balanced Accuracy is 0.5
Median Sensitivity is 0.0
Median Specificity is 1.0


,AUC,Balanced Accuracy,Sensitivity,Specificity
0,0.400000,0.500000,0.00,1.000000
1,0.236364,0.500000,0.00,1.000000
2,0.509091,0.654545,0.40,0.909091
3,0.500000,0.500000,0.00,1.000000
4,0.650000,0.625000,0.25,1.000000
5,0.581818,0.500000,0.00,1.000000
6,0.981818,0.500000,0.00,1.000000
7,0.818182,0.500000,0.00,1.000000
8,0.500000,0.575000,0.25,0.900000
9,0.275000,0.425000,0.25,0.600000


In [85]:
#MS high vs low impairment p-values
#run blocks 2-4 again
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests


df1 = df1.sort_values(by='EDSS')
df3 = df1.drop(['EDSS'], axis=1)
n,d = df3.shape

split_index = 0
for i in range(n):
    if df1.iat[i, d]==0:
        split_index +=1

print(split_index)
print(n)
low = df3.iloc[:split_index, :]
high = df3.iloc[split_index:, :]
print(p_val)

t_stat, p_val = ttest_ind(high, low, axis=0)
p_val_corrected = multipletests(p_val, method='fdr_bh')[1]
print(p_val_corrected)

53
76
[0.59240708 0.11824618 0.81325954 0.91352761 0.82666209 0.95675558
 0.00170995 0.84518411 0.88385203 0.83230889 0.07454485 0.87526975
 0.61223439 0.87773916 0.90312298 0.30611959 0.62331543 0.48732319
 0.32127161 0.59926756 0.70600486 0.14920906 0.90444481 0.66719231
 0.67275115 0.44367921 0.77326599 0.81570432 0.35731781 0.70444077
 0.33017851 0.7434726  0.10392987 0.22377983 0.14778813 0.16031202
 0.53514587 0.48773804 0.19059287 0.15646269 0.90714954 0.64573987
 0.55695395 0.92172126 0.65280426 0.87478652 0.5039277  0.09413599
 0.2861484  0.54537443 0.25006287 0.90409132 0.14926232 0.2079236
 0.56483545 0.14473503 0.56251461 0.57211957 0.71853032 0.24055728
 0.61937611 0.9110647  0.31852809 0.76383347 0.40203109 0.84968312
 0.07818166 0.05511145 0.38594739 0.08110311 0.19359402 0.38959886
 0.8781636  0.1320702  0.41510274 0.76975948 0.50742234 0.15531762
 0.10098051 0.17921344 0.6202803  0.42731985 0.74747236 0.83951516
 0.49535258 0.83292418]
[0.93256504 0.81099022 0.93256504

In [83]:
print(np.median(p_val))
print(np.median(p_val_corrected))

0.5511641896639541
0.9325650366731247
